In [ ]:
import sys

sys.path.append("/nfs/dust/belle2/user/kapitanl/Analysis/PRG_MPI/thumair_jpsiks/pythonTools")
sys.path.append("/nfs/dust/belle2/user/kapitanl/Analysis/PRD_MPI/Lib_Xgboost/xgboost/python-package")
!{sys.executable} -m pip install iminuit==2.2.1

%env TOERASEDIR=/nfs/dust/belle2/user/kapitanl/Analysis/Thesis/Hist/filesForRemove

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rootTools as tap
#import rootTools as tap

In [ ]:
import noteBookTools as nbt
totDicts = nbt.getDictForBDT()
listVars = [totDict['var'] for totDict in totDicts] + ['KFold', 'TagVLBoost', 'TagVmcLBoost', 'TagVLBoostErr']
listFormulas = [totDict['form'] for totDict in totDicts] + ['__event__ % 2', 'TagVLBoost', 'TagVmcLBoost', 'TagVLBoostErr']

totDictKins = nbt.getDictForBToDPi("BPtoJPsiKtoMuMu")
listVarKins = [totDictKin['var'] for totDictKin in totDictKins] #+ ['KFold']
listFormulaKins = [totDictKin['form'] for totDictKin in totDictKins] #+ ['KFold']

In [ ]:
fName = '/nfs/dust/belle2/user/kapitanl/Analysis/Thesis/Hist/datafortres.csv'
tupleDir = "/nfs/dust/belle2/user/jkandra/MC13-04/BPtoJPsiKtoMuMu/"


In [ ]:
#from root_pandas import read_root
#Data = read_root(tupleDir+"BPtoJPsiKtoMuMu.root", "BpToJPsiK_mumu")
Data = tap.getPandaFromFiles(fileNames=[tupleDir+"BPtoJPsiKtoMuMu.root"],
                               listVarFormulas=listFormulas + listFormulaKins,
                               listVarNames=listVars + listVarKins,
                               treeName="BpToJPsiK_mumu",cut="isSignal")


In [ ]:
vDatamc = Data["TagVmcLBoost"].to_numpy()
vData = Data["TagVLBoost"].to_numpy()
vDataRes = 1e4*np.subtract(vData,vDatamc)
vSig = (1e4*Data["TagVLBoostErr"]).to_numpy()

print(vData.size)
vData = vData[~np.isnan(vData)]
print(vData.size)


plt.hist(vDataRes, histtype='step', density = True, bins=np.linspace(-1, 1, 10))
print(vDataRes)

In [ ]:
def gausInt(a, b, c, d):
     m = d / c
     s = 1./np.sqrt(c)
     from math import erf
     res = np.sqrt(np.pi)/(2*np.sqrt(c)) * (erf((b*c-d)/np.sqrt(c)) - erf((a*c-d)/np.sqrt(c)))
     
     return res

def convExpGaus(sK, tau, x):
     c = 1./(2*sK*sK)
     d = 0.5*(x/sK/sK - 1/tau)
     Const = 1./(np.sqrt(2*np.pi) *sK* tau) * np.exp(-1./2 * pow(x/sK,2) + d*d/c)
     return Const * gausInt(0, 1e15, c, d);
  
def Gauss(x, mean, sigma):
    f = 1./(np.sqrt(2*np.pi)*sigma) * np.exp( -1./2 * pow((x-mean)/sigma, 2))
    return f

def fun(x, pars):
    fTMains = pars["fTMains"]
    fTR     = pars["fTR"]
    sigmas  = pars["sigmas"]
    cLMs    = pars["cLMs"]
    cRMs    = pars["cRMs"]
    mus     = pars["mus"]
    
    bigSigmaFrac = pars['bigSigmaFrac']
    fTBigs = pars['fTBigs']
    bigSigmaScale = pars['bigSigmaScale']
    cLBs = pars['cLBs']
    cRBs = pars['cRBs']
    
    ret = 0
    
    # small gauss
    ret += (1-bigSigmaFrac) * (1-fTMains) * Gauss(x, mean=mus, sigma=sigmas)
    ret += (1-bigSigmaFrac) * fTMains*(1.-fTR)*convExpGaus(sigmas, 1./cLMs, mus - x)
    ret += (1-bigSigmaFrac) * fTMains*(fTR)*convExpGaus(sigmas, 1./cRMs,  x - mus )
    # big gauss
    ret += bigSigmaFrac * (1-fTBigs) * Gauss(x, mean=mus, sigma=bigSigmaScale*sigmas)
    ret += bigSigmaFrac * fTBigs*(1.-fTR)*convExpGaus(bigSigmaScale*sigmas, 1./cLBs, mus - x)
    ret += bigSigmaFrac * fTBigs*(fTR)*convExpGaus(bigSigmaScale*sigmas, 1./cRBs,  x - mus )
    
    return ret




In [ ]:
import chebFitter as chf
xLim = 500
ch = chf.chebFitter(128 + 1, -xLim, xLim, vDataRes, fun)
#pars = {"mus" : 0, "sigmas" : 30,  "fTMains" : 0.9, "fTR" : 0.5, "fTBigs":0.1, "cLMs" : 0.1, "cRMs" : 0.1, "cLBs":0.1, "cRBs":0.1, "bigSigmaScale":5, "bigSigmaFrac":0.1 }
pars = {'mus': 4.8782559977170425, 'sigmas': 35.17551715919649,
        'fTMains': 0.2499141756210388, 'fTR': 0.5219611452793915,
        'fTBigs': 0.9999990463259867, 'cLMs': 0.011287758211660998, 
        'cRMs': 0.010676840203014085, 'cLBs': 0.00012179869239771871,
        'cRBs': 0.049979376629908555, 'bigSigmaScale': 4.745382823965359, 
        'bigSigmaFrac': 0.21782062995121432}
Limits = { 
        'mus': (-10, 10),
        'sigmas': (10, 40),
        'fTMains': (0, 1),
        'fTR': (0, 1),
        'cLMs': (1e-5, 0.1),
        'cRMs': (1e-5, 0.1),
         "bigSigmaScale": (3, 40),
        'bigSigmaFrac' : (0,1),
        'fTBigs' : (0,1),
        'cLBs' : (1e-5, 0.05),
        'cRBs' : (1e-5, 0.05)
        }


res =  ch.fitData(pars, Limits)
print(res)


In [ ]:
 
def plotHist(d, bins, chFit, res):
    vals, bins = np.histogram(d, bins)
    xCnt  = (bins[1:] + bins[:-1]) / 2
    xSize = (bins[1:] - bins[:-1]) / 2
    
    fig, axs = plt.subplots(2, sharex=True, gridspec_kw={'hspace': 0, 'height_ratios':[16,3]}, figsize=(12,8))
    axs[0].set_yscale('log')
    axs[0].errorbar(xCnt, vals, xerr=xSize, yerr=np.sqrt(vals), ls='none', fmt="o")
   # axs[0].legend(loc='upper right', shadow=True, fontsize='x-large')
    N = np.dot(vals, 2*xSize) 
    x = np.linspace(bins[0], bins[-1], 500)
    y = N * chFit.funFast(x, res)
    axs[0].plot(x, y, label='BPtoJPsiKtoMuMu')
    axs[0].legend(loc='upper right', shadow=True, fontsize='x-large')
    # plotRatio
    yRat= N * chFit.funFast(xCnt, res) 
    axs[1].errorbar(xCnt, vals/ yRat, xerr=xSize, yerr=np.sqrt(vals)/yRat, ls='none', fmt="o")
    #Save the figure as .png
    dirName = "/nfs/dust/belle2/user/kapitanl/Analysis/Thesis/Hist/"
    fig.savefig(dirName+"BPtoJPsiKtoMuMu_deltat1D.png", bbox_inches='tight')


    

In [ ]:
plotHist(vDataRes, np.linspace(-xLim, xLim, 200), ch, res)
